In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/sample_submission.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/AIMO_Progress_Prize_2_Reference_Problems_Solutions.pdf
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/aimo_2_gateway.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/__init__.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/aimo_2_inference_server.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/core/templates.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/core/base_gateway.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/core/relay.py
/kaggle/input/ai-mathematical-olympiad-progress-prize-2/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/ai-ma

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRITON_PTXAS_PATH"] = "/usr/local/cuda/bin/ptxas"
import re
import time
import random
import warnings
from collections import Counter
import numpy as np, pandas as pd, polars as pl

import torch
import vllm
from vllm import LLM, SamplingParams

import kaggle_evaluation.aimo_2_inference_server

warnings.simplefilter('ignore')
print('PyTorch version:', torch.__version__)
print('vLLM:', vllm.__version__)

INFO 03-16 12:37:24 __init__.py:183] Automatically detected platform cuda.
PyTorch version: 2.5.1+cu124
vLLM: 0.7.1


In [3]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
seed_everything(seed=0)

start_time = time.time()
cutoff_time = start_time + (4 * 60 + 45) * 60
cutoff_times = [int(x) for x in np.linspace(cutoff_time, start_time + 60 * 60, 50 + 1)]

In [4]:
if os.getenv('KAGGLE_KERNEL_RUN_TYPE') or os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    llm_model_pth = '/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b-awq-casperhansen/1'
else:
    llm_model_pth = '/root/volume/KirillR/QwQ-32B-Preview-AWQ'

MAX_NUM_SEQS = 32
MAX_MODEL_LEN = 8192 * 3 // 2

llm = LLM(
    llm_model_pth,
#    dtype="half",                 # The data type for the model weights and activations
    max_num_seqs=MAX_NUM_SEQS,    # Maximum number of sequences per iteration. Default is 256
    max_model_len=MAX_MODEL_LEN,  # Model context length
    trust_remote_code=True,       # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer
    tensor_parallel_size=4,       # The number of GPUs to use for distributed execution with tensor parallelism
    gpu_memory_utilization=0.95,  # The ratio (between 0 and 1) of GPU memory to reserve for the model
    seed=2024,
)

tokenizer = llm.get_tokenizer()

INFO 03-16 12:37:58 config.py:526] This model supports multiple tasks: {'embed', 'reward', 'classify', 'generate', 'score'}. Defaulting to 'generate'.
INFO 03-16 12:38:01 awq_marlin.py:109] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 03-16 12:38:01 config.py:1383] Defaulting to use mp for distributed inference
WARNING 03-16 12:38:01 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
INFO 03-16 12:38:01 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b-awq-casperhansen/1', speculative_config=None, tokenizer='/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b-awq-casperhansen/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=12288, download_dir=None, load_format=auto, tensor_parallel

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(VllmWorkerProcess pid=359) INFO 03-16 12:38:45 model_runner.py:1116] Loading model weights took 1.3375 GB
(VllmWorkerProcess pid=352) INFO 03-16 12:38:45 model_runner.py:1116] Loading model weights took 1.3375 GB
INFO 03-16 12:38:45 model_runner.py:1116] Loading model weights took 1.3375 GB
(VllmWorkerProcess pid=351) INFO 03-16 12:38:45 model_runner.py:1116] Loading model weights took 1.3375 GB
(VllmWorkerProcess pid=351) (VllmWorkerProcess pid=359) WARNING 03-16 12:39:02 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
WARNING 03-16 12:39:02 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
(VllmWorkerProcess pid=351) (VllmWorkerProcess pid=352) (VllmWorkerProcess pid=359) WARNING 03-16 12:39:02 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
WARNING 03-16 12:39:02 config.py:975] MLA is not supported with awq_marlin quantization. Disabling MLA.
WARNING 03-16 12:39:02 config.py:975] MLA is

Capturing CUDA graph shapes:   0%|          | 0/7 [00:00<?, ?it/s]

INFO 03-16 12:39:08 model_runner.py:1435] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-16 12:39:08 model_runner.py:1435] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-16 12:39:08 model_runner.py:1435] Capturing cudagraphs for decoding. This may lead to unexpected consequ

...

(VllmWorkerProcess pid=351) INFO 03-16 12:39:15 model_runner.py:1563] Graph capturing finished in 7 secs, took 0.10 GiB
(VllmWorkerProcess pid=352) 

Capturing CUDA graph shapes: 100%|██████████| 7/7 [00:07<00:00,  1.09s/it]

INFO 03-16 12:39:15 model_runner.py:1563] Graph capturing finished in 7 secs, took 0.10 GiB


Capturing CUDA graph shapes: 100%|██████████| 7/7 [00:07<00:00,  1.01s/it]

INFO 03-16 12:39:15 model_runner.py:1563] Graph capturing finished in 7 secs, took 0.10 GiB
(VllmWorkerProcess pid=359) INFO 03-16 12:39:15 model_runner.py:1563] Graph capturing finished in 7 secs, took 0.10 GiB
INFO 03-16 12:39:15 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 29.89 seconds


In [5]:
def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""

def batch_message_filter(list_of_messages) -> tuple[list[list[dict]], list[str]]:
    extracted_answers = []
    list_of_messages_to_keep = []
    for messages in list_of_messages:
        answer = extract_boxed_text(messages[-1]['content'])
        if answer:
            extracted_answers.append(answer)
        else:
            list_of_messages_to_keep.append(messages)
    return list_of_messages_to_keep, extracted_answers

def select_answer(answers):
    counter = Counter()
    for answer in answers:
        try:
            if int(answer) == float(answer):
                counter[int(answer)] += 1 + random.random() / 1_000
        except:
            pass
    if not counter:
        return 210
    _, answer = sorted([(v,k) for k,v in counter.items()], reverse=True)[0]
    return answer%1000

def batch_message_generate(list_of_messages) -> list[list[dict]]:
    max_tokens = MAX_MODEL_LEN
    if time.time() > cutoff_times[-1]:
        print("Speedrun")
        max_tokens = 2 * MAX_MODEL_LEN // 3

    sampling_params = SamplingParams(
        temperature=1.0,               # Randomness of the sampling
        top_p=0.90,                    # Cumulative probability of the top tokens to consider
        min_p=0.05,                    # Minimum probability for a token to be considered
        skip_special_tokens=True,      # Whether to skip special tokens in the output
        max_tokens=max_tokens,         # Maximum number of tokens to generate
        stop=["</think>"],             # List of strings that stop the generation
        seed=777,
    )
    
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]

    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    print([len(single_request_output.outputs[0].token_ids) for single_request_output in request_output])

    sort_keys_and_list_of_messages = []
    for messages, single_request_output in zip(list_of_messages, request_output):
        #print()
        #print(single_request_output.outputs[0].text)
        #print()
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})

        sort_keys_and_list_of_messages.append(
            (
                len(single_request_output.outputs[0].token_ids),
                messages
            )
        )
    print([sort_key for sort_key, _ in sort_keys_and_list_of_messages])
    sort_keys_and_list_of_messages.sort(key=lambda sort_key_and_messages: sort_key_and_messages[0])
    print([sort_key for sort_key, _ in sort_keys_and_list_of_messages])
    
    list_of_messages = [messages for _, messages in sort_keys_and_list_of_messages]
    return list_of_messages

In [6]:
def create_starter_messages(question, index):
    options = []
    for _ in range(13):
        options.append(
            [
                {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step. Return final answer within \\boxed{}, after taking modulo 1000."},
                {"role": "user", "content": question},
            ]
        )
    for _ in range(3):    
        options.append(
            [
                {"role": "system", "content": "You are a helpful and harmless assistant. You are Qwen developed by Alibaba. You should think step-by-step. After you get your final answer, take modulo 1000, and return the final answer within \\boxed{}."},
                {"role": "user", "content": question},
            ],
        )
    return options[index%len(options)]

def predict_for_question(question: str) -> int:
    selected_questions_only = True
    #selected_questions_only = False
    if selected_questions_only and not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        #if "Triangle" not in question:
        #    return 210
        if "Triangle" not in question and "delightful" not in question and "George" not in question:
            return 210

    if time.time() > cutoff_time:
        return 210
    
    print(question)

    num_seqs = MAX_NUM_SEQS
    if time.time() > cutoff_times[-1]:
        num_seqs = 2 * MAX_NUM_SEQS // 3
    
    list_of_messages = [create_starter_messages(question, index) for index in range(num_seqs)]

    all_extracted_answers = []
    for _ in range(1):
        list_of_messages = batch_message_generate(list_of_messages)
        
        if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
            df = pd.DataFrame(
                {
                    "question": [question] * len(list_of_messages),
                    "message": [messages[-1]["content"] for messages in list_of_messages],
                }
            )
            df.to_csv(f"{str(int(time.time() - start_time)).zfill(5)}.csv", index=False)
        
        list_of_messages, extracted_answers = batch_message_filter(list_of_messages)
        all_extracted_answers.extend(extracted_answers)
    
    print(all_extracted_answers)
    answer = select_answer(all_extracted_answers)
    print(answer)

    print("\n\n")
    cutoff_times.pop()
    return answer

def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    id_ = id_.item(0)
    print("------")
    print(id_)
    question = question.item(0)
    answer = predict_for_question(question)
    print(question)
    print("------\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [7]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [8]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
#            '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv',
            'reference.csv',
        )
    )

------
1acac0
Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?


Processed prompts: 100%|██████████| 32/32 [05:26<00:00, 10.19s/it, est. speed input: 10.06 toks/s, output: 674.91 toks/s]


[6039, 6653, 8467, 5939, 6545, 5848, 7684, 6513, 6292, 12188, 8449, 6203, 6927, 6721, 6952, 7482, 4481, 5502, 5502, 8004, 5878, 5254, 12188, 4967, 5740, 4850, 3828, 7529, 5392, 11455, 4326, 10344]
[6039, 6653, 8467, 5939, 6545, 5848, 7684, 6513, 6292, 12188, 8449, 6203, 6927, 6721, 6952, 7482, 4481, 5502, 5502, 8004, 5878, 5254, 12188, 4967, 5740, 4850, 3828, 7529, 5392, 11455, 4326, 10344]
[3828, 4326, 4481, 4850, 4967, 5254, 5392, 5502, 5502, 5740, 5848, 5878, 5939, 6039, 6203, 6292, 6513, 6545, 6653, 6721, 6927, 6952, 7482, 7529, 7684, 8004, 8449, 8467, 10344, 11455, 12188, 12188]
['180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180', '180']
180



Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?

Processed prompts: 100%|██████████| 32/32 [07:26<00:00, 13.96s/it, est. speed input: 10.78 toks/s, output: 862.59 toks/s]


[12070, 12140, 12140, 12140, 11057, 12140, 12140, 12140, 12140, 12140, 11269, 12140, 12140, 12132, 12132, 12132, 11791, 12140, 12140, 12140, 12140, 12140, 11373, 12140, 12140, 12140, 12140, 12140, 12140, 12132, 12132, 12132]
[12070, 12140, 12140, 12140, 11057, 12140, 12140, 12140, 12140, 12140, 11269, 12140, 12140, 12132, 12132, 12132, 11791, 12140, 12140, 12140, 12140, 12140, 11373, 12140, 12140, 12140, 12140, 12140, 12140, 12132, 12132, 12132]
[11057, 11269, 11373, 11791, 12070, 12132, 12132, 12132, 12132, 12132, 12132, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140, 12140]
['0', '2', '0', '0', '2']
0



We call a sequence $a_1, a_2, \ldots$ of non-negative integers \textit{delightful} if there exists a positive integer $N$ such that for all $n > N$, $a_n = 0$, and for all $i \geq 1$, $a_i$ counts the number of multiples of $i$ in $a_1, a_2, \ldots, a_N$. How many delightful sequences of non-n

Processed prompts: 100%|██████████| 32/32 [06:59<00:00, 13.12s/it, est. speed input: 10.02 toks/s, output: 874.06 toks/s]

[12159, 11959, 12159, 9458, 12159, 9956, 12159, 12159, 10420, 12159, 9610, 12159, 12159, 10802, 12151, 12151, 6768, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 6871, 12159, 12159, 12159, 12159, 12151, 12151, 11623]
[12159, 11959, 12159, 9458, 12159, 9956, 12159, 12159, 10420, 12159, 9610, 12159, 12159, 10802, 12151, 12151, 6768, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 6871, 12159, 12159, 12159, 12159, 12151, 12151, 11623]
[6768, 6871, 9458, 9610, 9956, 10420, 10802, 11623, 11959, 12151, 12151, 12151, 12151, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159, 12159]
['250', '250', '750', '0', '250', '750', '750', '0', '750']
750



Fred and George take part in a tennis tournament with $4046$ other players. In each round, the players are paired into $2024$ matches. How many ways are there to arrange the first round such that Fred and George do not have to play each other? (Two arrangements for the fir